In [1]:
import cv2,os
import torch,torchvision

In [2]:
from torchvision import models,transforms

In [3]:
transformation = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(112),
        transforms.RandomCrop(112),
        transforms.RandomRotation(180),
        transforms.RandomRotation(90),
        transforms.ColorJitter(brightness=0.5,contrast=0.5),
        transforms.ToTensor(),
    ]
)
truth = torch.tensor([0,1,0,0,0,1,1,1])
labels_reverse = {0:'not_intrested',1:'intrested'}

In [4]:
import pandas as pd
import numpy as np

In [5]:
from PIL import Image

In [6]:
def load_data(img_size=112):
    data = []
    index = -1
    labels = {}
    for label in os.listdir('./data/'):
        index += 1
        labels[f'./data/{label}/'] = [index,-1]
    if 'data.npy' in os.listdir('./'):
        return labels
    for label in labels:
        for file in os.listdir(label):
            try:
                file = label + file
                img = cv2.imread(file)
                img = cv2.resize(img,(img_size,img_size))
                data.append([
                    np.array(transformation(np.array(Image.fromarray(np.array(img))))),
                    labels[label][0]
                ])
                labels[label][1] += 1
            except Exception as e:
                print(e)
    for _ in range(25):
        np.random.shuffle(data)
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(d[1])
    VAL_SPLIT = int(len(data) * 0.03125)
    X_train = X[:-VAL_SPLIT]
    y_train = y[:-VAL_SPLIT]
    X_test = X[-VAL_SPLIT:]
    y_test = y[-VAL_SPLIT:]
    X_train = torch.tensor(X_train)
    X_test = torch.tensor(X_test)
    y_train = torch.tensor(y_train)
    y_test = torch.tensor(y_test)
    return [data,X,y,X_train,y_train,X_test,y_test,labels]

In [7]:
# data,X,y,X_train,y_train,X_test,y_test,labels = load_data()
# with-transformation
labels = load_data()

In [8]:
# np.save('./data.npy',data)
# # X_train,X_test,y_train,y_test = other_loading_data_proccess(data)
# np.save('./X_train.npy',np.array(X_train))
# np.save('./y_train.npy',np.array(y_train))
# np.save('./X_test.npy',np.array(X_test))
# np.save('./y_test.npy',np.array(y_test))

In [9]:
data = np.load('./data.npy',allow_pickle=True)
X_train = torch.from_numpy(np.load('./X_train.npy'))
y_train = torch.from_numpy(np.load('./y_train.npy'))
X_test = torch.from_numpy(np.load('./X_test.npy'))
y_test = torch.from_numpy(np.load('./y_test.npy'))

In [10]:
from tqdm import tqdm

In [11]:
IMG_SIZE = 112

In [12]:
labels

{'./data/no_like/': [0, -1], './data/like/': [1, -1]}

In [13]:
def accuracy(net, X, y,device='cpu',transformed=False):
    device = device
    net.to(device)
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        for i in tqdm(range(len(X))):
            real_class = y[i].to(device)
            if transformed:
                try:
                    net_out = net(
                        torch.tensor(np.array(transformation(np.array(Image.fromarray(np.array(X[i]*255).reshape(112,112,3).astype(np.uint8)))))).view(-1,3,IMG_SIZE,IMG_SIZE).to(device).float()
                    )
                except Exception as e:
                    print(e)
                    net_out = net(
                        torch.tensor(np.array(transformation(np.array(Image.fromarray(np.array(X[i])))))).view(-1,3,IMG_SIZE,IMG_SIZE).to(device).float()
                    )
            else:
                net_out = net(
                    X[i].view(-1,3,IMG_SIZE,IMG_SIZE).to(device).float()
                )
            preds = net_out[0][0]
            preds = float(preds)
            preds = round(preds)
            if preds == real_class:
                print(f'Pred : {preds}')
                print(f'Real : {real_class}')
                correct += 1
            total += 1
    net.train()
    print(correct)
    print(total)
    return round(correct / total, 3)

In [14]:
def accuracy_preds_out(preds, y):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(y))):
            real_class = y[i].to(device)
            pred = preds[i]
            pred = float(pred)
            pred = round(pred)
            if pred == real_class:
                print(f'Pred : {pred}')
                print(f'Real : {real_class}')
                correct += 1
            total += 1
    print(correct)
    print(total)
    return round(correct / total, 3)

In [15]:
def get_image(model='',shape=False,transformed=False,mode='eval',directory='./1st-try/test_data_old/',preds=False):
    imgs = []
    model.eval()
    preds_list = []
    idx = -1
    from PIL import Image
    import face_recognition
    with torch.no_grad():
        for path in os.listdir(directory):
            idx += 1
            path = f"{directory}/{path}"
            image = face_recognition.load_image_file(path)
            face_locations = face_recognition.face_locations(image)
            for fl in face_locations:
                color = (0, 0, 255)
                image_path = path
                image = cv2.imread(image_path)
            if len(face_locations) > 0:
                for face_location in face_locations:
                    im = Image.open(fr"{path}")
                    left = face_location[3]
                    top = face_location[0]
                    right = face_location[1]
                    bottom = face_location[2]
                    im1 = im.crop((left, top, right, bottom))
                    im1.save('./test.png')
                    img = cv2.imread('./test.png')
                    img = cv2.resize(img,(112,112))
                    imgs.append(img)
                    model.to(device)
                    if transformed:
                        preds = model(torch.tensor(np.array(transformation(np.array(Image.fromarray(np.array(img)))))).view(-1,3,112,112).to(device).float())
                    else:
                        preds = model(torch.tensor(img).view(-1,3,112,112).to(device).float())
                    preds = preds[0][0]
                    preds = float(preds)
                    preds = round(preds)
                    preds_list.append(preds)
                    plt.figure(figsize=(10,7))
                    plt.imshow(img)
                    plt.title(f'{labels_reverse[int(preds)]}')
                    plt.savefig(f'./{idx}.png')
                    plt.show()
    return torch.from_numpy(np.array(imgs))

In [16]:
import random

In [17]:
def best_pred(model='',shape=False,transformed=False,mode='eval',directory='./1st-try/test_data_old/',preds=False,num_of_iters=250):
    imgs = []
    model.eval()
    preds_dict = {}
    preds_all = []
    preds_all_final = []
    idx_dict = {}
    idx = -1
    from PIL import Image
    import face_recognition
    with torch.no_grad():
        for path in tqdm(os.listdir(directory)):
            path = f"{directory}/{path}"
            image = face_recognition.load_image_file(path)
            face_locations = face_recognition.face_locations(image)
            if len(face_locations) > 0:
                for face_location in face_locations:
                    idx += 1
                    for _ in range(num_of_iters):
                        im = Image.open(fr"{path}")
                        left = face_location[3]
                        top = face_location[0]
                        right = face_location[1]
                        bottom = face_location[2]
                        im1 = im.crop((left, top, right, bottom))
                        im1.save('./test.png')
                        img = cv2.imread('./test.png')
                        img = cv2.resize(img,(112,112))
                        imgs.append(img)
                        model.to(device)
                        if transformed:
                            preds = model(torch.tensor(np.array(transformation(np.array(Image.fromarray(np.array(img)))))).view(-1,3,112,112).to(device).float())
                        else:
                            preds = model(torch.tensor(img).view(-1,3,112,112).to(device).float())
                        preds = preds[0][0]
                        preds_all.append(preds)
                        preds = float(preds)
                        preds = round(preds)
                        idx_dict[idx] = img
                        try:
                            preds_dict[idx][preds]+=1
                        except Exception as e:
                            preds_dict[idx] = {0:-1,1:-1}
                            preds_dict[idx][preds]+=1
    for img,log in zip(preds_dict.keys(),preds_dict.values()):
        print(log)
        best_class = -1
        if log[0] < log[1] or log[1] > log[0]:
            print('1 is bigger than 0')
            best_class = 1
        elif log[0] > log[1] or log[1] < log[0]:
            print('0 is bigger than 1')
            best_class = 0
        else:
            best_class = random.choice([1,0])
        idx = img
        img = idx_dict[img]
        preds_all_final.append(best_class)
        plt.figure(figsize=(10,7))
        plt.imshow(img)
        plt.title(f'{labels_reverse[int(best_class)]}')
        plt.savefig(f'./{idx}.png')
        plt.show()
    return preds_dict,idx_dict,preds_all,preds_all_final

In [18]:
from torch import nn
import torch.nn.functional as F

In [19]:
class CNN_Test(nn.Module):
    def __init__(self,num_of_fc_layers=2,activation=F.relu,num_of_conv_layers=1,conv1_out=16,conv2_out=32,fc1_out=64,fc2_out=128):
        super().__init__()
        print(num_of_fc_layers)
        print(activation)
        print(num_of_conv_layers)
        print(conv1_out)
        print(conv2_out)
        print(fc1_out)
        print(fc2_out)
        self.pool = nn.MaxPool2d(2,2)
        self.conv1 = nn.Conv2d(3, conv1_out, 5)
        self.conv1batchnorm = nn.BatchNorm2d(conv1_out)
        self.conv2 = nn.Conv2d(conv1_out, conv2_out, 5)
        self.conv2batchnorm = nn.BatchNorm2d(conv2_out)
        self.conv3 = nn.Conv2d(conv2_out, conv2_out, 5)
        self.conv3batchnorm = nn.BatchNorm2d(conv2_out)
        self.fc1 = nn.Linear(conv2_out*10*10, fc1_out)
        self.fc1atchnorm = nn.BatchNorm1d(fc1_out)
        self.fc2 = nn.Linear(fc1_out, fc2_out)
        self.fc2atchnorm = nn.BatchNorm1d(fc2_out)
        self.fc4 = nn.Linear(fc2_out,fc2_out)
        self.fc4batchnorm = nn.BatchNorm1d(fc2_out)
        self.fc3 = nn.Linear(fc2_out, 1)
        self.activation = activation
        self.num_of_fc_layers = num_of_fc_layers
        self.conv2_out = conv2_out
        self.num_of_conv_layers = num_of_conv_layers

    def forward(self, x,shape=False):
        x = self.pool(self.activation(self.conv1batchnorm(self.conv1(x))))
        x = self.pool(self.activation(self.conv2batchnorm(self.conv2(x))))
        for _ in range(self.num_of_conv_layers):
            x = self.pool(torch.tensor(self.activation(self.conv3batchnorm(self.conv3(x)))))
        if shape:
            print(x.shape)
        x = x.view(-1, self.conv2_out*10*10)
        x = self.activation(self.fc1atchnorm(self.fc1(x)))
        x = self.activation(self.fc2atchnorm(self.fc2(x)))
        for _ in range(self.num_of_fc_layers):
            x = self.activation(self.fc4batchnorm(self.fc4(x)))
        x = torch.sigmoid(self.fc3(x))
        return x

In [20]:
device = 'cuda'

In [21]:
epochs = 125+31
# epochs = 200
# epochs = 250 + 125 + 31

In [22]:
import wandb
import matplotlib.pyplot as plt

In [23]:
BATCH_SIZE = 32

In [24]:
# model = CNN_Test().to(device)
# model.to(device)
# criterion = nn.BCELoss()
# optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
# wandb.init(project='Intred-or-Not-Product',name='test-cnn-without-transformation')
# for epoch in tqdm(range(epochs)):
#     torch.cuda.empty_cache()
#     for i in range(0,len(X_train),BATCH_SIZE):
#         optimizer.zero_grad()
#         torch.cuda.empty_cache()
#         X_batch = X_train[i:i+BATCH_SIZE].view(-1,3,112,112).to(device)
#         y_batch = y_train[i:i+BATCH_SIZE].to(device)
#         model.to(device)
#         preds = model(X_batch)
#         loss = criterion(preds.view(-1,1).float(),y_batch.view(-1,1).float())
#         loss.backward()
#         optimizer.step()
#         wandb.log({'loss_per_iter':loss.item()})
#     wandb.log({'loss_per_epoch':loss.item()})
# torch.cuda.empty_cache()
# test_acc_mean = np.mean([accuracy(model,get_image(model=model,shape=True,transformed=True,mode='eval'),truth,transformed=True),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=True)])
# wandb.log({'test_accuracy':test_acc_mean})
# torch.cuda.empty_cache()
# wandb.log({'val_accuracy':accuracy(model,X_test,y_test)})
# torch.cuda.empty_cache()
# wandb.log({'accuracy':accuracy(model,X_train[:1250],y_train[:1250])})
# torch.cuda.empty_cache()
# wandb.log({'val_accuracy':accuracy(model,X_test,y_test)})
# wandb.finish()

In [25]:
# 3 = lef
# 0 = top
# 1 = right
# 2 = bottom

In [26]:
class TL_Model_Test(nn.Module):
    def __init__(self,model=models.resnet18):
        super().__init__()
        self.model = model(pretrained=True)
#         self.model_v1_to_model_v2 = nn.Linear(1000,1204224)
#         self.model_v2 = model(pretrained=True)
#         self.output_1 = nn.Linear(1000,500)
#         self.relu = F.relu
#         self.output_1batchnorm = nn.BatchNorm1d(500)
#         self.output_2 = nn.Linear(500,750)
#         self.output_2batchnorm = nn.BatchNorm1d(750)
#         self.output_3 = nn.Linear(750,2)
        self.output = nn.Linear(1000,1)
    def forward(self,X):
        preds = self.model(X)
#         preds = self.model_v1_to_model_v2(preds)
#         preds = preds.view(X.shape)
#         preds = self.model_v2(preds)
#         preds = self.relu(self.output_1batchnorm(self.output_1(preds)))
#         preds = self.relu(self.output_2batchnorm(self.output_2(preds)))
#         preds = self.output_3(preds)
        preds = torch.sigmoid(self.output(preds))
        return preds

In [27]:
# model = TL_Model_Test().to(device)
# model.to(device)
# criterion = nn.BCELoss()
# optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
# wandb.init(project='Intred-or-Not-Product',name='test-tl_model-without-transformation')
# for epoch in tqdm(range(epochs)):
#     for i in range(0,len(X_train),BATCH_SIZE):
#         optimizer.zero_grad()
#         X_batch = X_train[i:i+BATCH_SIZE].view(-1,3,112,112).to(device)
#         y_batch = y_train[i:i+BATCH_SIZE].to(device)
#         model.to(device)
#         preds = model(X_batch)
#         preds.to(device)
#         loss = criterion(preds.view(-1,1).float(),y_batch.view(-1,1).float())
#         loss.backward()
#         optimizer.step()
#         wandb.log({'loss_per_iter':loss.item()})
#     wandb.log({'loss_per_epoch':loss.item()})
# torch.cuda.empty_cache()
# test_acc_mean = np.mean([accuracy(model,get_image(model=model,shape=True,transformed=True,mode='eval'),truth,transformed=True),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=True)])
# wandb.log({'test_accuracy':test_acc_mean})
# torch.cuda.empty_cache()
# wandb.log({'val_accuracy':accuracy(model,X_test,y_test)})
# torch.cuda.empty_cache()
# wandb.log({'accuracy':accuracy(model,X_train[:1250],y_train[:1250])})
# torch.cuda.empty_cache()
# wandb.log({'val_accuracy':accuracy(model,X_test,y_test)})
# wandb.finish()

In [28]:
# resnet18 = [models.resnet18,'resnet18']
# resnet34 = [models.resnet34,'resnet34']
# resnet50 = [models.resnet50,'resnet50']
# resnet101 = [models.resnet101,'resnet101']
# resnet152 = [models.resnet152,'resnet152']
# resnext50_32x4d = [models.resnext50_32x4d,'resnext50_32x4d']
# resnext101_32x8d = [models.resnext101_32x8d,'resnext101_32x8d']
# wide_resnet50_2 = [models.wide_resnet50_2,'wide_resnet50_2']
# wide_resnet101_2 = [models.wide_resnet101_2,'wide_resnet101_2']

In [29]:
# num_of_fc_layers = 
# activation
# num_of_conv_layers
# conv1_out
# conv2_out
# fc1_out
# fc2_out
# optimizer
# criterion
# img size
# epochs
# batch_size

In [30]:
# num_of_fc_layers = [1,2,3,4,5]
# for num_of_fc_layer in num_of_fc_layers:
#     model = Test_CNN(num_of_fc_layer=num_of_fc_layer).to(device)
#     model.to(device)
#     criterion = nn.BCELoss()
#     optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
#     wandb.init(project='Intred-or-Not-Product',name=f'{num_of_fc_layer}-num_of_fc_layer')
#     for epoch in tqdm(range(epochs)):
#         for i in range(0,len(X_train),BATCH_SIZE):
#             X_batch = X_train[i:i+BATCH_SIZE].view(-1,3,112,112).to(device)
#             y_batch = y_train[i:i+BATCH_SIZE].to(device)
#             model.to(device)
#             preds = model(X_batch)
#             preds.to(device)
#             loss = criterion(preds.view(-1,1).float(),y_batch.view(-1,1).float())
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             wandb.log({'loss_per_iter':loss.item()})
#         wandb.log({'loss_per_epoch':loss.item()})
#     test_acc_mean = np.mean([accuracy(model,get_image(model=model,shape=True,transformed=True,mode='eval'),truth,transformed=True),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=True)])
#     print(test_acc_mean)
#     torch.cuda.empty_cache()
#     wandb.log({'test_accuracy':test_acc_mean})
#     torch.cuda.empty_cache()
#     wandb.log({'val_accuracy':accuracy(model,X_test,y_test)})
#     torch.cuda.empty_cache()
#     wandb.log({'accuracy':accuracy(model,X_train[:1250],y_train[:1250])})
#     torch.cuda.empty_cache()
#     wandb.log({'val_accuracy':accuracy(model,X_test,y_test)})
#     wandb.finish()

In [31]:
alexnet = [models.alexnet,'alexnet']
squeezenet = [models.squeezenet1_0,'squeezenet1_0']
googlenet = [models.googlenet,'googlenet']
shufflenet = [models.shufflenet_v2_x1_0,'shufflenet_v2_x1_0']
resnet34 = [models.resnet34,'resnet34']
mobilenet_v2 = [models.mobilenet_v2,'mobilenet_v2']
mnasnet0_75 = [models.mnasnet0_75,'mnasnet0_75']
densenet161 = [models.densenet161,'densenet161']

In [32]:
epochs = 75

In [33]:
# # 
# models_ = [shufflenet,googlenet,squeezenet,[models.resnet18,'resnet18'],mobilenet_v2,densenet161,alexnet]
# for model in models_:
#     model_name = model
#     model = TL_Model_Test(model=model[0]).to(device)
#     model.to(device)
#     criterion = nn.BCELoss()
#     optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
#     wandb.init(project='Intred-or-Not-Product',name=f'{model_name[1]}-family')
#     for epoch in tqdm(range(epochs)):
#         for i in range(0,len(X_train),BATCH_SIZE):
#             optimizer.zero_grad()
#             X_batch = X_train[i:i+BATCH_SIZE].view(-1,3,112,112).to(device)
#             y_batch = y_train[i:i+BATCH_SIZE].to(device)
#             model.to(device)
#             preds = model(X_batch)
#             preds.to(device)
#             loss = criterion(preds.view(-1,1).float(),y_batch.view(-1,1).float())
#             loss.backward()
#             optimizer.step()
#             wandb.log({'loss_per_iter':loss.item()})
#         wandb.log({'loss_per_epoch':loss.item()})
#     torch.cuda.empty_cache()
#     test_acc_mean = np.mean([accuracy(model,get_image(model=model,shape=True,transformed=True,mode='eval'),truth,transformed=True),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=True)])
#     wandb.log({'test_accuracy':test_acc_mean})
#     torch.cuda.empty_cache()
#     val_acc_mean = np.mean([accuracy(model,X_test,y_test,transformed=True),accuracy(model,X_test,y_test,transformed=False)])
#     wandb.log({'val_accuracy':val_acc_mean})
#     torch.cuda.empty_cache()
#     train_acc_mean = np.mean([accuracy(model,X_train[:1250],y_train[:1250],transformed=True),accuracy(model,X_train[:1250],y_train[:1250],transformed=False)])
#     wandb.log({'accuracy':train_acc_mean})
#     wandb.finish()

In [34]:
# googlenet

In [35]:
# # torch.optim.Adadelta,torch.optim.Adagrad,torch.optim.Adam,torch.optim.AdamW,torch.optim.Adamax,torch.optim.ASGD,torch.optim.SGD
# optimizers = []
# for optimizer in optimizers:
#     model_name = model
#     model = TL_Model_Test(model=googlenet[0]).to(device)
#     model.to(device)
#     criterion = nn.BCELoss()
#     optimizer = optimizer(model.parameters(),lr=0.001)
#     wandb.init(project='Intred-or-Not-Product',name=f'{optimizer}-optimizer')
#     for epoch in tqdm(range(epochs)):
#         for i in range(0,len(X_train),BATCH_SIZE):
#             optimizer.zero_grad()
#             X_batch = X_train[i:i+BATCH_SIZE].view(-1,3,112,112).to(device)
#             y_batch = y_train[i:i+BATCH_SIZE].to(device)
#             model.to(device)
#             preds = model(X_batch)
#             preds.to(device)
#             loss = criterion(preds.view(-1,1).float(),y_batch.view(-1,1).float())
#             loss.backward()
#             optimizer.step()
#             wandb.log({'loss_per_iter':loss.item()})
#         wandb.log({'loss_per_epoch':loss.item()})
#     torch.cuda.empty_cache()
#     test_acc_mean = np.mean([accuracy(model,get_image(model=model,shape=True,transformed=True,mode='eval'),truth,transformed=True),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=True)])
#     wandb.log({'test_accuracy':test_acc_mean})
#     torch.cuda.empty_cache()
#     val_acc_mean = np.mean([accuracy(model,X_test,y_test,transformed=True),accuracy(model,X_test,y_test,transformed=False)])
#     wandb.log({'val_accuracy':val_acc_mean})
#     torch.cuda.empty_cache()
#     train_acc_mean = np.mean([accuracy(model,X_train[:1250],y_train[:1250],transformed=True),accuracy(model,X_train[:1250],y_train[:1250],transformed=False)])
#     wandb.log({'accuracy':train_acc_mean})
#     wandb.finish()

In [36]:
epochs = 37

In [37]:
# lrs = [1.0,0.1,0.01,0.001,0.0001]
# for lr in lrs:
#     model_name = model
#     model = TL_Model_Test(model=googlenet[0]).to(device)
#     model.to(device)
#     criterion = nn.BCELoss()
#     optimizer = torch.optim.Adamax(model.parameters(),lr=lr)
#     wandb.init(project='Intred-or-Not-Product',name=f'{lr}-lr')
#     for epoch in tqdm(range(epochs)):
#         for i in range(0,len(X_train),BATCH_SIZE):
#             optimizer.zero_grad()
#             X_batch = X_train[i:i+BATCH_SIZE].view(-1,3,112,112).to(device)
#             y_batch = y_train[i:i+BATCH_SIZE].to(device)
#             model.to(device)
#             preds = model(X_batch)
#             preds.to(device)
#             loss = criterion(preds.view(-1,1).float(),y_batch.view(-1,1).float())
#             loss.backward()
#             optimizer.step()
#             wandb.log({'loss_per_iter':loss.item()})
#         wandb.log({'loss_per_epoch':loss.item()})
#     torch.cuda.empty_cache()
#     test_acc_mean = np.mean([accuracy(model,get_image(model=model,shape=True,transformed=True,mode='eval'),truth,transformed=True),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=True)])
#     wandb.log({'test_accuracy':test_acc_mean})
#     torch.cuda.empty_cache()
#     val_acc_mean = np.mean([accuracy(model,X_test,y_test,transformed=True),accuracy(model,X_test,y_test,transformed=False)])
#     wandb.log({'val_accuracy':val_acc_mean})
#     torch.cuda.empty_cache()
#     train_acc_mean = np.mean([accuracy(model,X_train[:1250],y_train[:1250],transformed=True),accuracy(model,X_train[:1250],y_train[:1250],transformed=False)])
#     wandb.log({'accuracy':train_acc_mean})
#     wandb.finish()

In [38]:
# # nn.MSELoss,nn.L1Loss,torch.nn.HingeEmbeddingLoss,
# criterions = [torch.nn.MarginRankingLoss,nn.BCELoss]
# for criterion in criterions:
#     model = TL_Model_Test(model=googlenet[0]).to(device)
#     model.to(device)
#     criterion = criterion()
#     optimizer = torch.optim.Adamax(model.parameters(),lr=0.0001)
#     wandb.init(project='Intred-or-Not-Product',name=f'{criterion}-criterion')
#     for epoch in tqdm(range(epochs)):
#         for i in range(0,len(X_train),BATCH_SIZE):
#             optimizer.zero_grad()
#             X_batch = X_train[i:i+BATCH_SIZE].view(-1,3,112,112).to(device)
#             y_batch = y_train[i:i+BATCH_SIZE].to(device)
#             model.to(device)
#             preds = model(X_batch)
#             preds.to(device)
#             loss = criterion(preds.view(-1,1).float(),y_batch.view(-1,1).float())
#             loss.backward()
#             optimizer.step()
#             wandb.log({'loss_per_iter':loss.item()})
#         wandb.log({'loss_per_epoch':loss.item()})
#     torch.cuda.empty_cache()
#     test_acc_mean = np.mean([accuracy(model,get_image(model=model,shape=True,transformed=True,mode='eval'),truth,transformed=True),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=True)])
#     wandb.log({'test_accuracy':test_acc_mean})
#     torch.cuda.empty_cache()
#     val_acc_mean = np.mean([accuracy(model,X_test,y_test,transformed=True),accuracy(model,X_test,y_test,transformed=False)])
#     wandb.log({'val_accuracy':val_acc_mean})
#     torch.cuda.empty_cache()
#     train_acc_mean = np.mean([accuracy(model,X_train[:1250],y_train[:1250],transformed=True),accuracy(model,X_train[:1250],y_train[:1250],transformed=False)])
#     wandb.log({'accuracy':train_acc_mean})
#     wandb.finish()

In [39]:
# BCELoss = 2
# MSELoss = 3

In [40]:
# # 16,32,64,128,len(X_train)
# batch_sizes = [256,512,1024,2048]
# for batch_size in batch_sizes:
#     model = TL_Model_Test(model=googlenet[0]).to(device)
#     model.to(device)
#     criterion = nn.MSELoss()
#     optimizer = torch.optim.Adamax(model.parameters(),lr=0.0001)
#     wandb.init(project='Intred-or-Not-Product',name=f'{batch_size}-batch_size')
#     for epoch in tqdm(range(epochs)):
#         for i in range(0,len(X_train),batch_size):
#             optimizer.zero_grad()
#             X_batch = X_train[i:i+batch_size].view(-1,3,112,112).to(device)
#             y_batch = y_train[i:i+batch_size].to(device)
#             model.to(device)
#             preds = model(X_batch)
#             preds.to(device)
#             loss = criterion(preds.view(-1,1).float(),y_batch.view(-1,1).float())
#             loss.backward()
#             optimizer.step()
#             wandb.log({'loss_per_iter':loss.item()})
#         wandb.log({'loss_per_epoch':loss.item()})
#     torch.cuda.empty_cache()
#     test_acc_mean = np.mean([accuracy(model,get_image(model=model,shape=True,transformed=True,mode='eval'),truth,transformed=True),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=True)])
#     wandb.log({'test_accuracy':test_acc_mean})
#     torch.cuda.empty_cache()
#     val_acc_mean = np.mean([accuracy(model,X_test,y_test,transformed=True),accuracy(model,X_test,y_test,transformed=False)])
#     wandb.log({'val_accuracy':val_acc_mean})
#     torch.cuda.empty_cache()
#     train_acc_mean = np.mean([accuracy(model,X_train[:1250],y_train[:1250],transformed=True),accuracy(model,X_train[:1250],y_train[:1250],transformed=False)])
#     wandb.log({'accuracy':train_acc_mean})
#     wandb.finish()

In [41]:
# 128 = 1
# 16 = 1
# 64 = 1

In [42]:
# epochs = [37,50,100,125,250]
# for epoch in epochs:
#     model = TL_Model_Test(model=googlenet[0]).to(device)
#     model.to(device)
#     criterion = nn.MSELoss()
#     optimizer = torch.optim.Adamax(model.parameters(),lr=0.0001)
#     wandb.init(project='Intred-or-Not-Product',name=f'{epoch}-epoch')
#     for epoch in tqdm(range(epoch)):
#         for i in range(0,len(X_train),128):
#             optimizer.zero_grad()
#             X_batch = X_train[i:i+BATCH_SIZE].view(-1,3,112,112).to(device)
#             y_batch = y_train[i:i+BATCH_SIZE].to(device)
#             model.to(device)
#             preds = model(X_batch)
#             preds.to(device)
#             loss = criterion(preds.view(-1,1).float(),y_batch.view(-1,1).float())
#             loss.backward()
#             optimizer.step()
#             wandb.log({'loss_per_iter':loss.item()})
#         wandb.log({'loss_per_epoch':loss.item()})
#     torch.cuda.empty_cache()
#     test_acc_mean = np.mean([accuracy(model,get_image(model=model,shape=True,transformed=True,mode='eval'),truth,transformed=True),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=False),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=True)])
#     wandb.log({'test_accuracy':test_acc_mean})
#     torch.cuda.empty_cache()
#     val_acc_mean = np.mean([accuracy(model,X_test,y_test,transformed=True),accuracy(model,X_test,y_test,transformed=False)])
#     wandb.log({'val_accuracy':val_acc_mean})
#     torch.cuda.empty_cache()
#     train_acc_mean = np.mean([accuracy(model,X_train[:1250],y_train[:1250],transformed=True),accuracy(model,X_train[:1250],y_train[:1250],transformed=False)])
#     wandb.log({'accuracy':train_acc_mean})
#     wandb.finish()

In [43]:
# import cv2
# files = os.listdir('./test_data/')
# for file in files:
#     img = cv2.imread(f'./test_data/{file}',cv2.IMREAD_GRAYSCALE)
#     cv2.imwrite(f'./test_data/{file}',img)

In [44]:
configs = [googlenet,alexnet]
for config in configs:
    epochs = 125+31
    model = TL_Model_Test(config[0]).to(device)
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adamax(model.parameters(),lr=0.0001)
    wandb.init(project='Intred-or-Not-Product',name=f'{config[1]}-64')
    for epoch in tqdm(range(epochs)):
        for i in range(0,len(X_train),64):
            optimizer.zero_grad()
            X_batch = X_train[i:i+BATCH_SIZE].view(-1,3,112,112).to(device)
            y_batch = y_train[i:i+BATCH_SIZE].to(device)
            model.to(device)
            preds = model(X_batch)
            preds.to(device)
            loss = criterion(preds.view(-1,1).float(),y_batch.view(-1,1).float())
            loss.backward()
            optimizer.step()
            wandb.log({'loss_per_iter':loss.item()})
        wandb.log({'loss_per_epoch':loss.item()})
    torch.cuda.empty_cache()
    preds_dict,idx_dict,preds_all,preds_final = best_pred(model=model,shape=True,transformed=True,mode='eval',directory='./1st-try/test_data_old/',preds=True,num_of_iters=100)
    wandb.log({'test_accuracy_val_transformed':accuracy_preds_out(preds_final,truth)})
    preds_dict,idx_dict,preds_all,preds_final = best_pred(model=model,shape=True,transformed=False,mode='eval',directory='./1st-try/test_data_old/',preds=True,num_of_iters=100)
    wandb.log({'test_accuracy_val_not_transformed':accuracy_preds_out(preds_final,truth)})
    torch.cuda.empty_cache()
    wandb.log({'accuracy':accuracy(model,X_train[:1250],y_train[:1250])})
    torch.cuda.empty_cache()
    wandb.log({'val_accuracy':accuracy(model,X_test,y_test)})
    wandb.finish()
    best_pred(model=model,shape=True,transformed=True,mode='eval',directory='./1st-try/test_data_old/',preds=True)
    print('-'*50)
    best_pred(model=model,shape=True,transformed=False,mode='eval',directory='./1st-try/test_data_old/',preds=True) #88.4%

In [73]:
def grid_search_custom(configs={'model':[models.resnet18,models.resnet18],'batch_size':[128,64]}):
    final = []
    classes = list(configs.keys())
    

In [74]:
grid_search_custom()

['model', 'batch_size']


In [45]:
configs = [{'model':googlenet,'batch_size':128,'criterion':nn.BCELoss(),'optimizer':torch.optim.Adam,'lr':0.0001},{'model':alexnet,'batch_size':128,'criterion':nn.BCELoss(),'optimizer':torch.optim.Adam,'lr':0.0001}]
for config in configs:
    epochs = 125+31
    model = TL_Model_Test(config['model'][0]).to(device)
    model.to(device)
    criterion = config['criterion']
    optimizer = config['optimizer'](model.parameters(),lr=config['lr'])
    wandb.init(project='Intred-or-Not-Product',name=f'{config['model'][1]}-{config['batch_size']}-{config['criterion']}-{config['optimizer']}-{config['lr']}')
    for epoch in tqdm(range(epochs)):
        for i in range(0,len(X_train),config['batch_size']):
            optimizer.zero_grad()
            X_batch = X_train[i:i+BATCH_SIZE].view(-1,3,112,112).to(device)
            y_batch = y_train[i:i+BATCH_SIZE].to(device)
            model.to(device)
            preds = model(X_batch)
            preds.to(device)
            loss = criterion(preds.view(-1,1).float(),y_batch.view(-1,1).float())
            loss.backward()
            optimizer.step()
            wandb.log({'loss_per_iter':loss.item()})
        wandb.log({'loss_per_epoch':loss.item()})
    for _ in range(10):
        torch.cuda.empty_cache()
        preds_dict,idx_dict,preds_all,preds_final = best_pred(model=model,shape=True,transformed=True,mode='eval',directory='./1st-try/test_data_old/',preds=True,num_of_iters=125)
        wandb.log({'test_accuracy_val_transformed':accuracy_preds_out(preds_final,truth)})
        torch.cuda.empty_cache()
        preds_dict,idx_dict,preds_all,preds_final = best_pred(model=model,shape=True,transformed=False,mode='eval',directory='./1st-try/test_data_old/',preds=True,num_of_iters=100)
        wandb.log({'test_accuracy_val_not_transformed':accuracy_preds_out(preds_final,truth)})
        torch.cuda.empty_cache()
        test_acc_mean = np.mean([accuracy(model,get_image(model=model,shape=True,transformed=True,mode='eval'),truth,transformed=True),accuracy(model,get_image(model=model,shape=True,transformed=False,mode='eval'),truth,transformed=True)])
        wamdb.log('test_accuracy':test_acc_mean)                        
        torch.cuda.empty_cache()
        wandb.log({'val_accuracy':accuracy(model,X_test,y_test)})
    wandb.log({'accuracy':accuracy(model,X_train[:1250],y_train[:1250])})
    wandb.finish()

In [48]:
# epochs = 125+31
# model = TL_Model_Test(model=alexnet[0]).to(device)
# model.to(device)
# criterion = nn.MSELoss()
# optimizer = torch.optim.Adamax(model.parameters(),lr=0.0001)
# wandb.init(project='Intred-or-Not-Product',name=f'test')
# for epoch in tqdm(range(epochs)):
#     for i in range(0,len(X_train),128):
#         optimizer.zero_grad()
#         X_batch = X_train[i:i+BATCH_SIZE].view(-1,3,112,112).to(device)
#         y_batch = y_train[i:i+BATCH_SIZE].to(device)
#         model.to(device)
#         preds = model(X_batch)
#         preds.to(device)
#         loss = criterion(preds.view(-1,1).float(),y_batch.view(-1,1).float())
#         loss.backward()
#         optimizer.step()
#         wandb.log({'loss_per_iter':loss.item()})
#     wandb.log({'loss_per_epoch':loss.item()})
# torch.cuda.empty_cache()
# test_accuracy_val_transformed = []
# for _ in range(10):
#     preds_dict,idx_dict,preds_all,preds_final = best_pred(model=model,shape=True,transformed=True,mode='eval',directory='./1st-try/test_data_old/',preds=True,num_of_iters=100)
#     test_accuracy_val_transformed.append(accuracy_preds_out(preds_final,truth))
# wandb.log({'test_accuracy_val_transformed':np.mean(test_accuracy_val_transformed)})
# preds_dict,idx_dict,preds_all,preds_final = best_pred(model=model,shape=True,transformed=False,mode='eval',directory='./1st-try/test_data_old/',preds=True,num_of_iters=100)
# wandb.log({'test_accuracy_val_not_transformed':accuracy_preds_out(preds_final,truth)})
# torch.cuda.empty_cache()
# wandb.log({'accuracy':accuracy(model,X_train[:1250],y_train[:1250])})
# torch.cuda.empty_cache()
# wandb.log({'val_accuracy':accuracy(model,X_test,y_test)})
# wandb.finish()
# best_pred(model=model,shape=True,transformed=True,mode='eval',directory='./1st-try/test_data_old/',preds=True)
# print('-'*50)
# best_pred(model=model,shape=True,transformed=False,mode='eval',directory='./1st-try/test_data_old/',preds=True) #88.4%

In [ ]:
configs = [{'epochs':125+31},{'epochs':25},{'epochs':50},{'epochs':75},{'epochs':100},{'epochs':125},{'epochs':150},{'epochs':175},{'epochs':200},{'epochs':225},{'epochs':250},{'epochs':250+125},{'epochs':500}]
for config in configs:
    epochs = 125+31
    model = TL_Model_Test(model=googlenet[0]).to(device)
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adamax(model.parameters(),lr=0.0001)
    wandb.init(project='Intred-or-Not-Product',name=f'{config["epochs"]}')
    for epoch in tqdm(range(config['epochs'])):
        for i in range(0,len(X_train),128):
            optimizer.zero_grad()
            X_batch = X_train[i:i+BATCH_SIZE].view(-1,3,112,112).to(device)
            y_batch = y_train[i:i+BATCH_SIZE].to(device)
            model.to(device)
            preds = model(X_batch)
            preds.to(device)
            loss = criterion(preds.view(-1,1).float(),y_batch.view(-1,1).float())
            loss.backward()
            optimizer.step()
            wandb.log({'loss_per_iter':loss.item()})
        wandb.log({'loss_per_epoch':loss.item()})
    torch.cuda.empty_cache()
    test_accuracy_val_transformed = []
    for _ in range(10):
        preds_dict,idx_dict,preds_all,preds_final = best_pred(model=model,shape=True,transformed=True,mode='eval',directory='./1st-try/test_data_old/',preds=True,num_of_iters=100)
        test_accuracy_val_transformed.append(accuracy_preds_out(preds_final,truth))
    wandb.log({'test_accuracy_val_transformed':np.mean(test_accuracy_val_transformed)})
    preds_dict,idx_dict,preds_all,preds_final = best_pred(model=model,shape=True,transformed=False,mode='eval',directory='./1st-try/test_data_old/',preds=True,num_of_iters=100)
    wandb.log({'test_accuracy_val_not_transformed':accuracy_preds_out(preds_final,truth)})
    torch.cuda.empty_cache()
    wandb.log({'accuracy':accuracy(model,X_train[:1250],y_train[:1250])})
    torch.cuda.empty_cache()
    wandb.log({'val_accuracy':accuracy(model,X_test,y_test)})
    wandb.finish()
    best_pred(model=model,shape=True,transformed=True,mode='eval',directory='./1st-try/test_data_old/',preds=True)
    print('-'*50)
    best_pred(model=model,shape=True,transformed=False,mode='eval',directory='./1st-try/test_data_old/',preds=True) #88.4%

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


  3%|▎         | 5/156 [01:19<40:01, 15.91s/it]